# Homework 1

#### EE-556 Mathematics of Data - Fall 2022

If you worked with somebody else on this Homework, fill in their names here:

# Logistic Regression - 10 Points


We consider a binary classification task that we will model using logistic regression. Your goal will be to find a classifier using first-order methods and accelerated gradient descent methods. The first part will consist of more theoretical questions, and the second and third part will ask you to implement these methods.



Logistic regression is a classic approach to _binary classification_. Before we dive in, let us first define the standard logistic function $\sigma$ on which most of what follows is built:

\begin{equation*}
    \sigma : x \mapsto \frac{1}{1 + \exp{(-x)}}.
\end{equation*}

In logistic regression, we model the _conditional probability_ of observing a class label $b$ given a set of features $\mathbf{a}$. More formally, if we observe $n$ independent samples

\begin{equation*}
    \{(\mathbf{a}_i,b_i)\}_{i=1}^n,
\end{equation*}

where $\mathbf{a}_i\in\mathbb{R}^p$ and $b_i\in\{0, 1\}$ is the class label, we _assume_ that $b_i$ given $\mathbf{a}_i$ is a Bernouilli random variable with parameter $\sigma(\mathbf{a}_i^T\mathbf{x}^\natural)$, for some unknown $\mathbf{x}^\natural \in \mathbb{R}^p$. In other words, we assume that there exists an $\mathbf{x}^\natural \in \mathbb{R}^p$ such that

\begin{equation*}
    \mathbb{P}(b_i = 1 \mid \mathbf{a}_i) = \sigma(\mathbf{a}_i^T\mathbf{x}^\natural) \quad \text{ and } \quad \mathbb{P}(b_i = 0 \mid \mathbf{a}_i) = 1 - \sigma(\mathbf{a}_i^T\mathbf{x}^\natural)=  \sigma( - \mathbf{a}_i^T\mathbf{x}^\natural).
\end{equation*}

This is our statistical model. It can be written in a more compact form as follows,

\begin{equation*}
    \mathbb{P}(b_i = j \mid \mathbf{a}_i) = \sigma(\mathbf{a}_i^T\mathbf{x}^\natural)^j\sigma(-\mathbf{a}_i^T\mathbf{x}^\natural)^{1-j}, \quad j \in \{0, 1\}.
\end{equation*}

Our goal now is to determine the unknown $\mathbf{x}^\natural$ by constructing an estimator.

We are provided with a set of $n$ independent observations, we can write down the negative log-likelihood $f$ as follows:

\begin{equation*}
	\begin{aligned}
		f(\mathbf{x}) = -\log(\mathbb{P}(b_1, \dots, b_n | a_1, \dots, a_n)) & = -\log \left( \prod_{i=1}^{n} \mathbb{P}(b_i|a_i) \right) \quad \quad \text{(by independence)}\\
		 & =  \sum_{i=1}^n - \log(\sigma(\mathbf{a}_i^T\mathbf{x})^{b_i}\sigma(-\mathbf{a}_i^T\mathbf{x})^{1-b_i}) \\
		 & = \sum_{i=1}^n  b_i \log(1 + \exp(- \mathbf{a}_i^T\mathbf{x})) + (1-b_i)\log(1 + \exp(\mathbf{a}_i^T\mathbf{x})).
	\end{aligned}
\end{equation*}


__(a)__ (1 point) Show that the function $u \mapsto \log(1 + \exp(u))$ is convex. Deduce that $f$ is convex.

$\textbf{Ans.}$ We can show that $f(u) = \log(1+\exp(u))$ is a convex function by showing that $f''(u) \geq 0$ for all u.

\begin{equation*}
\begin{aligned}
    f'(u) &= \frac{\exp(u)}{1+\exp(u)} \\
    &= 1 - \frac{1}{1+\exp(u)}\\
   f''(u) &= \frac{\exp(u)}{(1+\exp(u))^2} \geq 0
    \end{aligned}
 \end{equation*}
We have the last inequality based ont he fact that the $\exp(u)$ is always non-negative

You have just established that the negative log-likelihood is a convex function. So in principle, any local minimum of the maximum likelihood estimator, which is defined as

\begin{equation*}
    \mathbf{x}^\star_{ML} = \arg\min_{\mathbf{x} \in \mathbb{R}^p} f(\mathbf{x}),
\end{equation*}

is a global minimum so it can serve as an estimator of $\mathbf{x}^\natural$. But, does the minimum always exist? We will ponder this question in the following three points.

__(b)__ (1 point) Explain the difference between infima and minima.  Give an example of a convex function, defined over $\mathbb{R}$, that does not attain its infimum. 

$\textbf{Ans.}$ Minima of a function is the lowest value attained by the function. Infima is the greatest value that is either less than or equal to all the values attained by the function. This means that infima may not neccessarily be attained hy the function, whereas the minima is always attained by the function.
$\newline$
An example of convex function that does not attain its infimum is $f(x) = \exp(-x)$. The infimum of this function is 0, but this value is never attained for any finite value of x.

__(c)__ (1 points) Let us assume that there exists $\mathbf{x}_0 \in \mathbb{R}^p$ such that 

\begin{equation*}
    \forall i\in\{1, \dots, n\}, \quad \quad \text{ if } b_i = 1, \text{ then } \mathbf{a}_i^T\mathbf{x}_0 > 0, \text{ and if } b_i = 0 \text{ then } \mathbf{a}_i^T\mathbf{x}_0 < 0.
\end{equation*}

This is called _complete separation_ in the literature. Can you think of a geometric reason why this name is appropriate? Think of a 2D example where this can happen (i.e $p=2$) and describe why _complete separation_ is an appropriate name.



$\textbf{Ans.}$ The name is appropriate because the existence of such an $x_0$ implies that there exists a plane that perfectly separates the datapoints of the two classes [0,1]. In the case of 2D, $x_0$ can be interpreted as a vector along the line perpendicular to the line separating the datatpoints of the 2 classes. This condition also states that there are no datapoints on the line separating the classes thus implying complete separation.

From this, you should see that it is likely that our data satisfies the complete separation assumption. Unfortunately, as you will show in the following question, this can become an obstacle.

__(d)__ (2 points) In a _complete separation_ setting, i.e, there exists $\mathbf{x}_0$ such that 

\begin{equation*}
    \forall i\in\{1, \dots, n\}, \quad \quad \text{ if } b_i = 1, \text{ then } \mathbf{a}_i^T\mathbf{x}_0 > 0, \text{ and if } b_i = 0 \text{ then } \mathbf{a}_i^T\mathbf{x}_0 < 0.
\end{equation*}

prove that the function $f$ does not attain its minimum. 

__Hint__: If the function did have a minimum, would it be above, below or equal to zero? Then think of how $f(2 \mathbf{x}_0)$ compares with $f(\mathbf{x}_0)$, how about $f(\alpha \mathbf{x}_0)$ for $\alpha \rightarrow + \infty$ ? 

$\textbf{Ans.}$ Given that there exists an $x_0$ and we have the complete separation setting, then we can divide the datapoints based on the class labels $b_i$. Let j be the indices for class 1 and k be the indices for class 0. Then we can thus write down f(x) as 
\begin{equation*}
\begin{aligned}
 f(x) & = \sum_{i=1}^n  b_i \log(1 + \exp(- \mathbf{a}_i^T\mathbf{x})) + (1-b_i)\log(1 + \exp(\mathbf{a}_i^T\mathbf{x})).\\
 & = \sum_{j}  \log(1 + \exp(- \mathbf{a}_j^T\mathbf{x})) + \sum_{k} \log(1 + \exp(\mathbf{a}_k^T\mathbf{x})) &\geq 0.
\end{aligned}
\end{equation*}
Using the fact that in the complete separation setting that $\mathbf{a}_j^T\mathbf{x}_0 > 0$ and  $\mathbf{a}_k^T\mathbf{x}_0 < 0$ we have the following expression for $f(x)$. We observe that f(x) is a non-negative decreasing function of x. Now if we assume that $f(x)$ had a minima at $x = x^*$ then we can easily show that $f(2 x^*) \le f(x^*)$ by susbtituting in the expression derived above. Using contradiction we thus have that $f(x)$ never attains its minimum and has an infima = 0.

As you have just shown convex functions do not always attain their infimum. So it is possible for the maximum-likelihood estimator $\mathbf{x}^\star_{ML}$ to not exist. We will resolve this issue by adding a regularizer. 

In what follows, we consider the function

\begin{equation*}
	f_\mu(\mathbf{x}) = f(\mathbf{x}) + \frac{\mu}{2}\|\mathbf{x}\|_2^2
\end{equation*}
with $\mu> 0$.

__(e)__ (1 points) Show that the gradient of $f_\mu$ can be expressed as 
\begin{equation}
	\nabla f_\mu(\mathbf{x}) = \sum_{i=1}^n (\sigma(\mathbf{a}_i^T\mathbf{x}) - b_i)\mathbf{a}_i + \mu \mathbf{x}.
\end{equation}
__Hint__: Lecture 3 shows you how to proceed with this question.

$\textbf{Ans.}$ Using the exrpression above, we calculate the exopression for $\nabla f_\mu(\mathbf{x})$ as follows,

\begin{equation*}
\begin{aligned}
	\nabla f_\mu(\mathbf{x}) &= \nabla f(\mathbf{x}) + \frac{\mu}{2}\nabla \mathbf{x^Tx}.\\
     &= \nabla f(\mathbf{x}) + \mu\mathbf{x}.
\end{aligned}
\end{equation*}
Now we calculate $\nabla f_\mu(\mathbf{x})$ using the expression given earlier in the description -
\begin{equation*}
\begin{aligned}
	\nabla f(\mathbf{x}) &= \nabla \left(\sum_{i=1}^n  b_i \log(1 + \exp(- \mathbf{a}_i^T\mathbf{x})) +  + (1-b_i)\log(1 + \exp(\mathbf{a}_i^T\mathbf{x}))\right).\\
    &= -\sum_{i=1}^n b_i\frac{ a_i \exp(- \mathbf{a}_i^T\mathbf{x})}{1 + \exp(- \mathbf{a}_i^T\mathbf{x})} + \sum_{i=1}^n (1-b_i)\frac{ a_i \exp(\mathbf{a}_i^T\mathbf{x})}{1 + \exp(\mathbf{a}_i^T\mathbf{x})} \\
    &= \sum_{i=1}^n  \left(-b_i \mathbf{a}_i(1-\sigma(\mathbf{a}_i^T\mathbf{x})) + (1-b_i)\mathbf{a}_i \sigma(\mathbf{a}_i^T\mathbf{x})\right)\\
    &= \sum_{i=1}^n \left(-b_i \mathbf{a}_i+ b_i \mathbf{a}_i\sigma(\mathbf{a}_i^T\mathbf{x})) + \mathbf{a}_i \sigma(\mathbf{a}_i^T\mathbf{x})- b_i\mathbf{a}_i \sigma(\mathbf{a}_i^T\mathbf{x})\right)\\
    &= \sum_{i=1}^n  ( \sigma(\mathbf{a}_i^T\mathbf{x}) - b_i)\mathbf{a}_i
\end{aligned}
\end{equation*}
Substituting the form of $\nabla f(x)$ into the expression for $\nabla f_{\mu}(x)$ we show that - 
\begin{equation*}
	\nabla f_\mu(\mathbf{x}) = \sum_{i=1}^n (\sigma(\mathbf{a}_i^T\mathbf{x}) - b_i)\mathbf{a}_i + \mu \mathbf{x}.
\end{equation*}

__(f)__ (1 points) Show that the Hessian of $f_\mu$ can be expressed as 
\begin{equation}
	\nabla^2 f_\mu(\mathbf{x}) = \sum_{i=1}^{n} \sigma(\mathbf{a}_i^T\mathbf{x})\sigma(- \mathbf{a}_i^T\mathbf{x})\mathbf{a}_i\mathbf{a}_i^T + \mu \mathbf{I}.
\end{equation}




$\textbf{Ans.}$ Previously we have shown that 
$$
    \nabla f_\mu(\mathbf{x}) = \sum_{i=1}^n (\sigma(\mathbf{a}_i^T\mathbf{x}) - b_i)\mathbf{a}_i + \mu \mathbf{x}.
$$
Taking another derivate we get:

$$
\begin{aligned}
    \nabla^2 f_\mu(\mathbf{x}) &= \nabla\bigg\{\sum_{i=1}^n (\sigma(\mathbf{a}_i^T\mathbf{x}) - b_i)\mathbf{a}_i + \mu \mathbf{x}\bigg\}.\\
    &= \sum_{i=1}^n \frac{\exp(-a_i^Tx)\mathbf{a}_i \mathbf{a}_i^T}{(1+\exp(-a_i^Tx))^2} + \mu \mathbf{I}. \\
    &= \sum_{i=1}^n \frac{1}{(1+\exp(-a_i^Tx))}\frac{\exp(-a_i^Tx)}{(1+\exp(-a_i^Tx))}\mathbf{a}_i \mathbf{a}_i^T + \mu \mathbf{I}. \\
    &= \sum_{i=1}^{n} \sigma(\mathbf{a}_i^T\mathbf{x})\sigma(- \mathbf{a}_i^T\mathbf{x})\mathbf{a}_i\mathbf{a}_i^T + \mu \mathbf{I}.
\end{aligned}
$$
Thus we show that the hessian of $f_{\mu}$ is $\nabla^2 f_\mu(\mathbf{x}) = \sum_{i=1}^{n} \sigma(\mathbf{a}_i^T\mathbf{x})\sigma(- \mathbf{a}_i^T\mathbf{x})\mathbf{a}_i\mathbf{a}_i^T + \mu \mathbf{I}.$

It is convenient to observe that we can write the Hessian in a more compact form by defining the matrix
\begin{equation}
	\mathbf{A} = \begin{bmatrix}
        \leftarrow &  \mathbf{a}_1^T & \rightarrow \\
        \leftarrow &  \mathbf{a}_2^T & \rightarrow \\
         &  \ldots &  \\
        \leftarrow &  \mathbf{a}_n^T & \rightarrow \\
  \end{bmatrix}.
\end{equation}
It is easy to see that we have
\begin{equation}
	\nabla^2 f_\mu(\mathbf{x}) =  \mathbf{A}^T \text{Diag}\left( \sigma(\mathbf{a}_i^T\mathbf{x})\sigma(- \mathbf{a}_i^T\mathbf{x}) \right)\mathbf{A}+ \mu \mathbf{I}.
\end{equation}


__(g)__ (1 point) Show that $f_\mu$ is $\mu$-strongly convex. 

$\textbf{Ans.}$ As shown above,
\begin{equation*}
\begin{aligned}
	\nabla^2 f_\mu(\mathbf{x}) - \mu \mathbf{I}. &=  \mathbf{A}^T \text{Diag}\left( \sigma(\mathbf{a}_i^T\mathbf{x})\sigma(- \mathbf{a}_i^T\mathbf{x}) \right)\mathbf{A}\\
    &=  \sum_{i=1}^{n} \mathbf{a_i}^T \text{Diag}\left( \sigma(\mathbf{a}_i^T\mathbf{x})\sigma(- \mathbf{a}_i^T\mathbf{x}) \right)\mathbf{a_i}^T
\end{aligned}
\end{equation*}
Using the fact that $\text{Diag}\left( \sigma(\mathbf{a}_i^T\mathbf{x})\sigma(- \mathbf{a}_i^T\mathbf{x})\right)$ is a positive definite, we have that $\nabla^2 f_\mu(\mathbf{x}) -  \mu \mathbf{I}$ is convex, implying that $f_\mu(\mathbf{x})$ is $\mu$- strongly convex.

__(h)__ (1 point) Is it possible for a strongly convex function, defined over $\mathbb{R}^p$, to not attain its minimum ? <a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1) Justify your reasoning.

$\textbf{Ans.}$  No, it is not possible for a strongly convex function to not attain its minimum. If a function f(x) is $\mu$-strongly convex function then we can express it as 
$$
    f(\mathbf{x}) = g(\mathbf{x})  + \frac{\mu}{2}\|x\|^2_2
$$
where g(x) is a convex function.$\newline$ We have shown earlier that a convex function can have an infima and it tends towards the infima at the extremes of its domain i.e. $\|x\|_2 \to \infty$. Now if we study the behavior of f(x) we see that even if g(x) has an infima, the term $\frac{\mu}{2}\|x\|^2_2$ makes it tends towards $\infty$:  as $\lim_{\|x\|^2_2 \to +\infty} f(x) = \infty$. Thus we have no infima for f(x) and the minimum of f(x) is attained at a finite $\mathbf{x_{min}}$ such that $\nabla f(\mathbf{x_{min}}) = \nabla g(\mathbf{x_{min}})  + \mu\mathbf{x_{min}} = 0$, since both extremes of the domain form a maximum point for f(x).

We will now show that $f_\mu$ is smooth, i.e, $\nabla f_\mu$ is L-Lipschitz with respect to the Euclidean norm, with 
\begin{equation}
	L = \|A\|^2_F + \mu \text{, where }\|\cdot\|_F\text{ denotes the Frobenius norm. }
\end{equation}

(1 point for all three questions)

__(i-1)__ Show that $\lambda_{\max}(\mathbf{a}_i\mathbf{a}_i^T) = \left\| \mathbf{a}_i\right\|_2^2$, where $\lambda_{\max}(\cdot)$ denotes the largest eigenvalue.

We define matrix M such that $M = \mathbf{a_i}\mathbf{a_i^T}$, and d = Dim($\mathbf{a_i}$). In the case of M we observe that there will be d-1 vectors that are orthogonal to $\mathbf{a_i}$ that form a set B. This means that if $\mathbf{b}\in B$ then Mb=0. Now we have that the vectors in set B are eigenvectors of M with eigenvalue = 0  and $\mathbf{a_i}$ is the only vector with a non-zero eigenvalue. Since $M\mathbf{a_i} = \mathbf{a_i}\mathbf{a_i}^T\mathbf{a_i} = \|\mathbf{a_i}\|_2^2\mathbf{a_i}$. Our eigenvalues are $(\|\mathbf{a_i}\|_2^2, 0, 0.....0)$. Thus $\lambda_{\max}(\mathbf{a}_i\mathbf{a}_i^T) = \left\| \mathbf{a}_i\right\|_2^2$.

__(i-2)__ Using [2](#mjx-eqn-eq2), show that $\lambda_{\max}(\nabla^2 f_\mu(\mathbf{x})) \leq \sum_{i=1}^{n} \|\mathbf{a}_i\|_2^2 + \mu$. 

__Hint__: Recall that $\lambda_{\max}(\cdot)$ verifies the triangle inequality.

Using the relation we obtained earlier 
$$
    \lambda_{\max}(\nabla^2 f_\mu(\mathbf{x})) = \lambda_{\max}(\sum_{i=1}^{n} \sigma(\mathbf{a}_i^T\mathbf{x})\sigma(- \mathbf{a}_i^T\mathbf{x})\mathbf{a}_i\mathbf{a}_i^T + \mu \mathbf{I}).
$$
Using the fact that $\sigma(x) < 1$ for finite x, we have
$$
    \lambda_{\max}(\sum_{i=1}^{n} \sigma(\mathbf{a}_i^T\mathbf{x})\sigma(- \mathbf{a}_i^T\mathbf{x})\mathbf{a}_i\mathbf{a}_i^T + \mu \mathbf{I}) \leq \sum_{i=1}^{n} \lambda_{\max}(\mathbf{a}_i\mathbf{a}_i^T) + \mu .
$$
Substituting the the result $\lambda_{\max}(\mathbf{a}_i\mathbf{a}_i^T) = \left\| \mathbf{a}_i\right\|_2^2$ we get,
$$
    \lambda_{\max}(\nabla^2 f_\mu(\mathbf{x})) \leq \sum_{i=1}^{n} \left\| \mathbf{a}_i\right\|_2^2 + \mu.
$$
Hence proved

__(i-3__ Conclude that $f_\mu$ is $L$-smooth for $L = \|A\|_F^2 + \mu$.

We have shown earlier that $\lambda_{\max}(\nabla^2 f_\mu(\mathbf{x})) \leq \sum_{i=1}^{n} \left\| \mathbf{a}_i\right\|_2^2 + \mu =  \|A\|_F^2 + \mu.\newline$ This means that the matrix S = $\nabla^2 f_\mu(\mathbf{x}) -  (\|A\|_F^2 + \mu)\mathbf{I} $ has eigenvalues are non-positive. Thus we can conclude that S is a negative semi-definite matrix. This is similar to saying that $\nabla^2 f_\mu(\mathbf{x}) \preccurlyeq L\mathbf{I}$, which the required condiiton for L-smoothness. Thus we conclude $f_\mu$ is $L$-smooth for $L = \|A\|_F^2 + \mu$..


From your work in this section, you have shown that the maximum likelihood estimator for logistic regression might not exist, but it can be guaranteed to exist by adding a $\|\cdot\|_2^2$ regularizer. Consequently, the estimator for $\mathbf{x}^\natural$ we will use will be the solution of the smooth strongly convex problem,
\begin{equation}
	\mathbf{x}^\star=\arg\min_{\mathbf{x} \in \mathbb{R}^p} f(\mathbf{x}) + \frac{\mu}{2}\|\mathbf{x}\|_2^2.
\end{equation}

<a name="cite_note-1"></a>1. [^](#cite_ref-1) TA's will give you candy if you provide a complete proof.